In [1]:
###custom model

In [81]:
### import modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
layers = keras.layers

In [82]:
### the model 
def get_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation='relu')(inputs)
    dropout  = layers.Dropout(0.5)(features)
    output   = layers.Dense(10, activation = 'softmax')(dropout)
    
    model = keras.Model(inputs, output)
    return model

In [83]:
### define loss and optimizers and metrics
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
loss_tracking_mean = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

In [84]:
model = get_model()

In [85]:
### training_step

@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training = True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    logs = {}
    
    for metric in metrics:
        metric.update_state(targets, predictions)
        result = metric.result()
        logs[metric.name] = result
        
    loss_tracking_mean.update_state(loss)
    logs['loss'] = loss_tracking_mean.result()
    
    return logs

In [86]:
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_mean.reset_state()

In [93]:
## the data 
(train_images, train_labels) , (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28*28)).astype("float32") / 255
test_images  = test_images.reshape((10000, 28*28)).astype("float32") / 255

training_dataset = tf.data.Dataset.from_tensor_slices((
train_images, train_labels))

validation_dataset = tf.data.Dataset.from_tensor_slices((
test_images, test_labels))

training_dataset = training_dataset.batch(32)
validation_dataset = validation_dataset.batch(32)
#train_images, train_labels = train_images[:10000] , train_labels[:10000]

In [89]:
keras.backend.clear_session()
epochs = 3
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch , targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"Training Result At Epoch {epoch} :")
    for key, value in logs.items():
        print(f"...{key} :  {value:.4f}")
        

Training Result At Epoch 0 :
...sparse_categorical_accuracy :  0.9208
...loss :  0.2680
Training Result At Epoch 1 :
...sparse_categorical_accuracy :  0.9575
...loss :  0.1577
Training Result At Epoch 2 :
...sparse_categorical_accuracy :  0.9650
...loss :  0.1348


In [97]:
## eval
def eval_model(inputs, targets):
    predictions = model(inputs, training = False)
    loss = loss_fn(targets, predictions)
    
    logs = {}
    
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs['val_'+metric.name] = metric.result()
    loss_tracking_mean.update_state(loss)
    
    logs['val_loss'] = loss_tracking_mean.result()
    return logs

In [99]:
reset_metrics()
for inputs_batch , targets_batch in validation_dataset:
    logs = eval_model(inputs_batch ,targets_batch)
    
for key , value in logs.items():
    print(f'...{key}  ::  {value:.4f}')

...val_sparse_categorical_accuracy  ::  0.9722
...val_loss  ::  0.1196


### Subclassing the model Object

In [114]:
class CustomModel(tf.keras.Model):
    def train_step(self, inputs):
        features, targets = inputs
        with tf.GradientTape() as tape:
            predictions = model(features)
            loss = self.compiled_loss(targets, predictions)
        gradients = tape.gradient(loss, model.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        

        self.compiled_metrics.update_state(targets, predictions)
        return {m.name : m.result() for m in self.metrics}
        
        

In [115]:
### the model 
def get_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation='relu')(inputs)
    dropout  = layers.Dropout(0.5)(features)
    output   = layers.Dense(10, activation = 'softmax')(dropout)
    
    model = CustomModel(inputs, output)
    return model

In [116]:
model = get_model()

In [117]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = 'RMSprop',
             metrics = ['accuracy'])

In [118]:
model.fit(training_dataset, epochs = 3)

Epoch 1/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.1949 - accuracy: 0.9431
Epoch 2/3
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0890 - accuracy: 0.9751
Epoch 3/3
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0636 - accuracy: 0.9831


In [120]:
model.evaluate(validation_dataset)

313/313 [==============================] - 1s 3ms/step - loss: 0.1174 - accuracy: 0.9697


[0.1174054816365242, 0.9696999788284302]